In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pytpc
from pytpc.constants import degrees, pi

In [ ]:
pt = pytpc.Particle(4, 2, 2, position=(0,0,1.0), polar=pi-10*degrees, azimuth=28*degrees)
gas = pytpc.gases.InterpolatedGas('helium', 200.)
ef = np.array([0, 0, 15e3])
bf = np.array([0, 0, 1])

vd = pytpc.simulation.drift_velocity_vector(-2, ef[2], bf[2], 0)

simres = pytpc.track(pt, gas, ef, bf)
simres[['x', 'y', 'z']] *= 1e3
evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values, simres['de'].values, 6.25, vd, 40.2, 
                               pt.mass_num, 223, 190)

In [ ]:
vd

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
plt.plot(simres.y, simres.x)

In [ ]:
pytpc.pad_plot(evt.hits());

In [ ]:
meas = evt.xyzs(vd, 6.25, peaks_only=True)[:, 0:3] * 1e-3
act = simres[['x', 'y', 'z', 'px', 'py', 'pz']].copy()
act[['x', 'y', 'z']] *= 1e-3

In [ ]:
pt = pytpc.Particle(4, 2, 2, position=meas[10], polar=pi-10*degrees, azimuth=28*degrees)
tr = pytpc.Tracker(pt, gas, ef, bf, pt.state_vector)
tr.kfilter.Q = np.diag((1e-4, 1e-4, 1e-4, 1e-1, 1e-1, 1e-1))**2 # Process
tr.kfilter.R = np.diag([2e-2]*2 + [4e-2]) ** 2 # Measurement
tr.kfilter.P = np.diag([5e-2] * 3 + [1e-0] * 3)**2

res, covar, res_times = tr.track(meas)

In [ ]:
pytpc.tpcplot.state_vector_plots(x_act=simres.time.values, act=act.values,
                                 x_calc=res_times, calc=res, x_data=res_times, data=meas, covar=covar);

In [ ]:
np.diff(res_times).mean()

In [ ]:
res_sm, covar_sm = tr.kfilter.smooth(res, covar, res_times)

In [ ]:
pytpc.tpcplot.state_vector_plots(x_act=simres.time.values, act=act.values,
                                 x_calc=res_times, calc=res_sm, x_data=res_times, data=meas, covar=covar_sm);